In [1]:
%xmode plain
import glob
import cv2
import numpy as np
import torch.nn as nn
import torch.nn.functional as f
import torch
import torch.optim as optim

Exception reporting mode: Plain


In [2]:
suits_dict = {0:'Spade',1:'Heart', 2:'Club', 3:'Diamond'}
ranks_dict = {0:'A', 1:'2', 2:'3', 3:'4', 4:'5', 5:'6', 6:'7', 7:'8', 8:'9', 9:'10', 10:'J', 11:'K', 12:'Q'}
BKG_THRESH = 60
CARD_THRESH = 30

In [3]:
def flattener(image, pts, w, h):
    """Flattens an image of a card into a top-down 200x300 perspective.
    Returns the flattened, re-sized, grayed image.
    See www.pyimagesearch.com/2014/08/25/4-point-opencv-getperspective-transform-example/"""
    temp_rect = np.zeros((4,2), dtype = "float32")
    
    s = np.sum(pts, axis = 2)

    tl = pts[np.argmin(s)]
    br = pts[np.argmax(s)]
    
    diff = np.diff(pts, axis = -1)
    tr = pts[np.argmin(diff)]
    bl = pts[np.argmax(diff)]
    
#     pri
    # Need to create an array listing points in order of
    # [top left, top right, bottom right, bottom left]
    # before doing the perspective transform

    if w <= 0.8*h: # If card is vertically oriented
        temp_rect[0] = tl
        temp_rect[1] = tr
        temp_rect[2] = br
        temp_rect[3] = bl

    if w >= 1.2*h: # If card is horizontally oriented
        temp_rect[0] = bl
        temp_rect[1] = tl
        temp_rect[2] = tr
        temp_rect[3] = br


    if w > 0.8*h and w < 1.2*h: #If card is diamond oriented

        if pts[1][0][1] <= pts[3][0][1]:
            # If card is titled to the left, approxPolyDP returns points
            # in this order: top right, top left, bottom left, bottom right
            temp_rect[0] = pts[1][0] # Top left
            temp_rect[1] = pts[0][0] # Top right
            temp_rect[2] = pts[3][0] # Bottom right
            temp_rect[3] = pts[2][0] # Bottom left

        # If furthest left point is lower than furthest right point,
        # card is tilted to the right
        if pts[1][0][1] > pts[3][0][1]:
            # If card is titled to the right, approxPolyDP returns points
            # in this order: top left, bottom left, bottom right, top right
            temp_rect[0] = pts[0][0] # Top left
            temp_rect[1] = pts[3][0] # Top right
            temp_rect[2] = pts[2][0] # Bottom right
            temp_rect[3] = pts[1][0] # Bottom left
            
        
    maxWidth = 200
    maxHeight = 300

    # Create destination array, calculate perspective transform matrix,
    # and warp card image
    dst = np.array([[0,0],[maxWidth-1,0],[maxWidth-1,maxHeight-1],[0, maxHeight-1]], np.float32)
    M = cv2.getPerspectiveTransform(temp_rect,dst)
    warp = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

    return warp

In [4]:
def preprocess_card(contour, img):
    
    peri = cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, 0.01*peri, True)
    pts = np.float32(approx)
    x,y,w,h = cv2.boundingRect(approx)
    
    warp = flattener(img, pts, w, h)
    
    ## Card image
    warp = cv2.cvtColor(warp,cv2.COLOR_BGR2GRAY)

    CORNER_WIDTH = 30
    CORNER_HEIGHT = 84
    
    cropped_suit = warp[0:CORNER_HEIGHT, 0:CORNER_WIDTH]
    csuit = cv2.resize(cropped_suit, (0,0), fx=4, fy=4)
    
    white_level = csuit[15,int((CORNER_WIDTH*4)/2)]
    thresh_level = white_level - 30 # CARD_THRESH
    if (thresh_level <= 0):
        thresh_level = 1
        
    retval, csuit = cv2.threshold(csuit, thresh_level, 255, cv2. THRESH_BINARY_INV)
    
    
    half = int(csuit.shape[0]/2)
        
    return csuit[0:half,:] , csuit[half+1:half*2,:]

In [5]:
class Net(nn.Module):
    def __init__(self, num_inputs=6000, num_classes=4):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(num_inputs, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = f.relu(self.fc1(x))
        x = f.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
predict_suits = Net()
predict_suits.load_state_dict(torch.load('./suits-model-1538689727.9620585.csv'))
predict_suits.eval()
predict_ranks = Net(num_classes=13)
predict_ranks.load_state_dict(torch.load('./ranks-model-1538687613.6711135.csv'))
predict_ranks.eval()

Net(
  (fc1): Linear(in_features=6000, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=13, bias=True)
)

In [21]:
# Loop through the cards in the image and classify them
mask = cv2.imread('./CV-BlackjackStudent/GOPR0318.MP4-lbl/00445-lbl.png',1)
input_img = cv2.imread('./CV-BlackjackStudent/GOPR0318.MP4-lbl/00445.png',1)
mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(mask,128,255,0)
_, input_img_t = cv2.threshold(input_img,128,255,0)
im2, num_cards, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)


for i in range(len(num_cards)):
    
    card_value, card_suit = preprocess_card(num_cards[i], input_img)

    im2, contours, hierarchy = cv2.findContours(card_value,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea,reverse=True)
    
    x, y, w, h = cv2.boundingRect(contours[0])
    isolated_rank = card_value[y:y+h, x:x+w]
    im2, contours, hierarchy = cv2.findContours(card_suit,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea,reverse=True)
    
    x, y, w, h = cv2.boundingRect(contours[0])
    isolated_suit = card_suit[y:y+h, x:x+w]

    ## Read in the card values
    isolated_suit = cv2.resize(isolated_suit, (60, 100))
    isolated_rank = cv2.resize(isolated_rank, (60, 100))

    isolated_suit = isolated_suit.flatten()
    isolated_rank = isolated_rank.flatten()
    a = predict_suits(torch.tensor(isolated_suit, dtype=torch.float32)).max(0)[1].item()
    b = predict_ranks(torch.tensor(isolated_rank, dtype=torch.float32)).max(0)[1].item() #find_rank(isolated_rank, ranks)
 
    ## Put text on screen
    text = "Card: " +  ranks_dict[b]+ ' of ' + suits_dict[a] +"s" 
    cv2.putText(input_img,  text, (50, (i+1)*100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), thickness=2,lineType=cv2.LINE_AA) 
  
cv2.imshow('Scene',input_img)
cv2.waitKey(0)
cv2.destroyAllWindows()